In [1]:
import os
import uuid
import numpy as np 
import pandas as pd
#import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
# This is used in order to show the plotted figures within this notebook
%matplotlib inline 
# This is used in order to show the tensorboard within this notebook             
#%load_ext tensorboard  
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import Counter
from math import ceil
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_roc_curve
from sklearn.model_selection import learning_curve
from sklearn.multioutput import MultiOutputClassifier
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from evaluate import run
import datetime


In [2]:
data = pd.read_csv('output/loan_dev.csv')
data

,Id,loan_date,loan_duration,loan_payments,Predicted,account_frequency,account_district_region,account_district_no_inhabitants,account_district_no_municipalities_0_499,account_district_no_municipalities_500_1999,...,last_trans_balance,mean_trans_balance,mean_trans_amount_absolute,mean_trans_amount_credit,mean_trans_amount_withdrawal,mean_trans_amount_signed,owner_male,owner_age,account_age_months,has_disponent
0,5314,930705,12,8033,1.0,1,7,94812,15,13,...,20100.0,12250.000000,5025.000000,5025.000000,NaN,5025.000000,0,45,3,0
1,5316,930711,36,4610,0.0,2,3,112709,48,20,...,52208.9,52083.859459,11015.635135,13523.158824,8884.240000,1411.051351,1,24,4,0
2,6863,930728,60,2118,0.0,2,3,77917,85,19,...,20272.8,30060.954167,5417.458333,5009.733333,6097.000000,844.708333,1,57,5,0
3,5325,930803,36,2939,0.0,2,2,107870,84,29,...,34307.3,41297.480000,8253.080000,9254.600000,7168.100000,1371.704000,0,53,6,0
4,7240,930906,60,4579,0.0,1,4,1204953,0,0,...,41112.9,57188.211111,18945.966667,21255.930769,16801.000000,1522.707407,1,14,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,961212,48,3242,0.0,2,1,226122,32,50,...,60694.1,44197.509884,4895.974419,6448.582857,3830.458824,352.872093,0,25,22,1
324,5625,961215,60,3703,1.0,2,7,45714,52,10,...,59578.8,55230.444068,11725.494915,13417.557143,10197.180645,1009.813559,1,57,12,0
325,6805,961221,48,938,0.0,2,1,285387,0,2,...,38384.3,41994.907692,7779.815385,8544.930000,6974.431579,984.215385,0,39,7,0
326,7233,961225,36,3217,0.0,2,6,93931,74,21,...,41878.1,56646.516129,13280.084677,16554.986275,10992.139726,337.726613,1,43,19,0


# Manual Outlier Detection

- 

In [7]:
for col in data.columns:
    print("-"*50)
    print(f"Statistics of {col}")
    print("-"*50)
    #data[col].plot.box();

    q3, q1 = np.percentile(data[col], [75 ,25])

    print("Min: ", end="");                 print(data[col].min())
    print("Max: ", end="");                 print(data[col].max())
    print("Mean: ", end="");                print(data[col].mean())
    print("Median: ", end="");              print(data[col].median())
    print("Standard deviation: ", end="");  print(data[col].std())
    print("Skewness: ", end="");            print(data[col].skew())  
    print("Variance: ", end="");            print(data[col].var())
    print("Kurtosis: ", end="");            print(data[col].kurtosis())
    print("Interquartile Range: ", end=""); print(q3 - q1, end=" | "); print("Q1: {} | Q3: {}".format(q1,q3))

# var() method returns the unbiased variance
# Medium-High std which indicates that the data is spread out and not clustered around the mean
# Skewness > 1 => it is skewed to the right

--------------------------------------------------
Statistics of Id
--------------------------------------------------
Min: 4959
Max: 7308
Mean: 6205.658536585366
Median: 6227.5
Standard deviation: 667.985674726219
Skewness: -0.126359735421936
Variance: 446204.86163944204
Kurtosis: -1.168537695082133
Interquartile Range: 1133.0 | Q1: 5604.25 | Q3: 6737.25
--------------------------------------------------
Statistics of loan_date
--------------------------------------------------
Min: 930705
Max: 961227
Mean: 949989.125
Median: 950565.5
Standard deviation: 9495.504646353156
Skewness: -0.2574182118951275
Variance: 90164608.48891437
Kurtosis: -1.1466194893417647
Interquartile Range: 19716.0 | Q1: 940809.25 | Q3: 960525.25
--------------------------------------------------
Statistics of loan_duration
--------------------------------------------------
Min: 12
Max: 60
Mean: 35.853658536585364
Median: 36.0
Standard deviation: 16.73475166172723
Skewness: 0.05589223008522359
Variance: 280.05191

In [22]:
outliers = {}
for col in data.columns:
    data_mean, data_std = data[col].mean(), data[col].std()
    # identify outliers
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off 
    outliers[col] = pd.concat([data.loc[data[col] < lower] , (data.loc[data[col] > upper])])

for key in outliers.keys():
    if (len(outliers[key]) > 0):
        print("-"*50)
        print(f"Outliers of {key} - mean: {data[key].mean()} - std: {data[key].std()}")
        print("-"*50)
        print(outliers[key][key])
        

--------------------------------------------------
Outliers of account_frequency - mean: 1.7347560975609757 - std: 0.5527904760035754
--------------------------------------------------
14     0
15     0
28     0
43     0
61     0
70     0
111    0
160    0
172    0
196    0
197    0
207    0
220    0
251    0
277    0
307    0
321    0
322    0
Name: account_frequency, dtype: int64
--------------------------------------------------
Outliers of account_district_no_municipalities_0_499 - mean: 39.24085365853659 - std: 34.85013667376864
--------------------------------------------------
52     151
54     151
86     151
171    151
248    151
Name: account_district_no_municipalities_0_499, dtype: int64
--------------------------------------------------
Outliers of account_district_no_municipalities_2000_9999 - mean: 5.432926829268292 - std: 4.441438312314091
--------------------------------------------------
112    20
Name: account_district_no_municipalities_2000_9999, dtype: int64
--------

In [26]:
outliers_ids = set([val for key in outliers.keys() for val in outliers[key]['Id']])
outlier_ratio = len(outliers_ids) / len(data)
outlier_ratio

0.25914634146341464

In [32]:
data

,Id,loan_date,loan_duration,loan_payments,Predicted,account_frequency,account_district_region,account_district_no_inhabitants,account_district_no_municipalities_0_499,account_district_no_municipalities_500_1999,...,last_trans_balance,mean_trans_balance,mean_trans_amount_absolute,mean_trans_amount_credit,mean_trans_amount_withdrawal,mean_trans_amount_signed,owner_male,owner_age,account_age_months,has_disponent
0,5314,930705,12,8033,1.0,1,7,94812,15,13,...,20100.0,12250.000000,5025.000000,5025.000000,NaN,5025.000000,0,45,3,0
1,5316,930711,36,4610,0.0,2,3,112709,48,20,...,52208.9,52083.859459,11015.635135,13523.158824,8884.240000,1411.051351,1,24,4,0
2,6863,930728,60,2118,0.0,2,3,77917,85,19,...,20272.8,30060.954167,5417.458333,5009.733333,6097.000000,844.708333,1,57,5,0
3,5325,930803,36,2939,0.0,2,2,107870,84,29,...,34307.3,41297.480000,8253.080000,9254.600000,7168.100000,1371.704000,0,53,6,0
4,7240,930906,60,4579,0.0,1,4,1204953,0,0,...,41112.9,57188.211111,18945.966667,21255.930769,16801.000000,1522.707407,1,14,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,961212,48,3242,0.0,2,1,226122,32,50,...,60694.1,44197.509884,4895.974419,6448.582857,3830.458824,352.872093,0,25,22,1
324,5625,961215,60,3703,1.0,2,7,45714,52,10,...,59578.8,55230.444068,11725.494915,13417.557143,10197.180645,1009.813559,1,57,12,0
325,6805,961221,48,938,0.0,2,1,285387,0,2,...,38384.3,41994.907692,7779.815385,8544.930000,6974.431579,984.215385,0,39,7,0
326,7233,961225,36,3217,0.0,2,6,93931,74,21,...,41878.1,56646.516129,13280.084677,16554.986275,10992.139726,337.726613,1,43,19,0


In [29]:
manual_outlier_df = data.loc[~data['Id'].isin(outliers_ids)]
manual_outlier_df

,Id,loan_date,loan_duration,loan_payments,Predicted,account_frequency,account_district_region,account_district_no_inhabitants,account_district_no_municipalities_0_499,account_district_no_municipalities_500_1999,...,last_trans_balance,mean_trans_balance,mean_trans_amount_absolute,mean_trans_amount_credit,mean_trans_amount_withdrawal,mean_trans_amount_signed,owner_male,owner_age,account_age_months,has_disponent
1,5316,930711,36,4610,0.0,2,3,112709,48,20,...,52208.9,52083.859459,11015.635135,13523.158824,8884.240000,1411.051351,1,24,4,0
2,6863,930728,60,2118,0.0,2,3,77917,85,19,...,20272.8,30060.954167,5417.458333,5009.733333,6097.000000,844.708333,1,57,5,0
3,5325,930803,36,2939,0.0,2,2,107870,84,29,...,34307.3,41297.480000,8253.080000,9254.600000,7168.100000,1371.704000,0,53,6,0
4,7240,930906,60,4579,0.0,1,4,1204953,0,0,...,41112.9,57188.211111,18945.966667,21255.930769,16801.000000,1522.707407,1,14,6,0
5,6687,930913,24,3660,0.0,2,2,53921,61,22,...,43847.8,46318.552941,9496.929412,7331.985714,19600.000000,2579.282353,0,13,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,5388,961128,12,5130,0.0,1,7,51313,34,12,...,20051.1,32967.712676,9739.397183,16171.554545,6851.489796,282.411268,1,53,10,0
320,5591,961204,48,3242,0.0,2,2,92084,55,29,...,24447.1,39426.070000,9151.153333,9343.175000,8931.700000,814.900000,0,37,7,0
324,5625,961215,60,3703,1.0,2,7,45714,52,10,...,59578.8,55230.444068,11725.494915,13417.557143,10197.180645,1009.813559,1,57,12,0
326,7233,961225,36,3217,0.0,2,6,93931,74,21,...,41878.1,56646.516129,13280.084677,16554.986275,10992.139726,337.726613,1,43,19,0


# Automatic Outlier Detection

In [37]:
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope

In [41]:
import sys
import os

In [45]:
df = pd.read_csv(
    f'output\\loan_full_dataset.csv', sep=";")
df.head()


,loan_id;account_id;date;amount;duration;payments;status
0,5314;1787;930705;96396;12;8033;B
1,5316;1801;930711;165960;36;4610;A
2,6863;9188;930728;127080;60;2118;A
3,5325;1843;930803;105804;36;2939;A
4,7240;11013;930906;274740;60;4579;A


## Isolation Forest

In [38]:
# identify outliers in the training dataset
iso = IsolationForest()
yhat = iso.fit_predict(data)
mask = yhat != -1
no_outlier_iso = data[mask, :]
no_outlier_iso

ValueError: Input X contains NaN.
IsolationForest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# Dataset without Outliers